# Spotify Sync

In [3]:
import requests
import pandas as pd
from typing import List, Dict, Any

## Environment variables

In [38]:
SPOTIFY_API = <SPOTIFY_API>
OAUTH_TOKEN_USER_1 = <OAUTH_TOKEN_USER_1>
OAUTH_TOKEN_USER_2 = <OAUTH_TOKEN_USER_2>

## Get User's Saved Tracks

In [ ]:
def get_user_saved_tracks(offset: int) -> List[Dict[str, Any]]:
  try:
    headers = {
        'Authorization': f'Bearer {OAUTH_TOKEN_USER_1}'
    }
    r = requests.get(f'{SPOTIFY_API}/me/tracks?limit=50&offset={offset}', headers=headers)
    r.raise_for_status()
    return r.json()
  except requests.exceptions.HTTPError as error:
    return f'Error to get user saved tracks: {error}'

In [ ]:
def filter_user_saved_tracks() -> List[Dict[str, Any]]:
  current_tracks = [None]
  saved_tracks = []
  offset = 0
  while len(current_tracks) != 0:
    data = get_user_saved_tracks(offset)
    df = pd.DataFrame(data)
    df = df.filter(['items'])
    df = pd.json_normalize(df.to_dict('records'))
    df = df.filter(['items.track.id', 'items.track.name', 'items.track.external_urls.spotify'])
    df.rename(columns={'items.track.id': 'track_id', 'items.track.name': 'track_name', 'items.track.external_urls.spotify': 'track_url'}, inplace=True)
    current_tracks = [track for track in df.to_dict('records')]
    for saved_track in current_tracks:
      saved_tracks.append(saved_track)
    offset += 50
  return saved_tracks

## Save Tracks for User

In [ ]:
def save_tracks_for_user(ids: str) -> None:
  try:
    headers = {
        'Authorization': f'Bearer {OAUTH_TOKEN_USER_2}'
    }
    r = requests.put(f'{SPOTIFY_API}/me/tracks?ids={ids}', data={}, headers=headers)
    r.raise_for_status()
  except requests.exceptions.HTTPError as error:
    return f'Error to save tracks for user: {error}'

In [ ]:
def filter_tracks_for_user() -> None:
  saved_tracks = filter_user_saved_tracks()
  df_saved_tracks = pd.DataFrame(saved_tracks)
  saved_tracks_list = df_saved_tracks['track_id'].tolist()
  offset = 0
  while offset < len(saved_tracks_list):
    ids_list = [track for track in saved_tracks_list[offset:offset+50]]
    ids = ",".join(ids_list)
    save_tracks_for_user(ids)
    offset += 50

In [ ]:
filter_tracks_for_user()

## Get a List of Current User's Playlists

In [44]:
def get_user_playlists(oauth_token: str, offset: int) -> any:
    try:
        headers = {
            'Authorization': f'Bearer {oauth_token}'
        }
        r = requests.get(f'https://api.spotify.com/v1/me/playlists?limit=50&offset={offset}', headers=headers)
        r.raise_for_status()
        return r.json()
    except requests.exceptions.HTTPError as error:
        return f'Error to get user playlists: {error}'

In [45]:
def filter_user_playlists(oauth_token: str) -> list:
    try:
        current_playlists = [None]
        saved_playlists = []
        offset = 0
        while len(current_playlists) != 0:
            data = get_user_playlists(oauth_token, offset)
            df = pd.DataFrame(data)
            df = df.filter(['items'])
            df = pd.json_normalize(df.to_dict('records'))
            df = df.filter(['items.id'])
            current_playlists = [playlist for playlist in df.to_dict('records')]
            for saved_playlist in current_playlists:
                saved_playlists.append(saved_playlist)
            offset += 50
        print('User playlists have been filtered successfully')
        return saved_playlists
    except Exception as error:
        print(f'Error to filter user playlists: {error}')
        raise

## Follow a Playlist

In [47]:
def follow_playlist(oauth_token: str, playlist_id: int) -> any:
    try:
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {oauth_token}'
        }
        r = requests.put(f'https://api.spotify.com/v1/playlists/{playlist_id}/followers', data={}, headers=headers)
        r.raise_for_status()
        return r.status_code
    except requests.exceptions.HTTPError as error:
        return f'Error to follow playlist: {error}'

In [48]:
def filter_playlists_for_user(oauth_token_1: str, oauth_token_2: str) -> None:
    try:
        saved_playlists = filter_user_playlists(oauth_token_1)
        df_saved_playlists = pd.DataFrame(saved_playlists)
        saved_playlists_list = df_saved_playlists['items.id'].tolist()
        for playlist_id in saved_playlists_list:
            status_code = follow_playlist(oauth_token_2, playlist_id)
            if status_code == 204:
                time.sleep(1)
        print(f'Playlists for user have been followed successfully')
    except Exception as error:
        print(f'Error to filter playlists for user: {error}')
        raise